In [ ]:
!pip install diffusers transformers torch accelerate geographiclib

In [ ]:

import torch
import json
import numpy as np
import pandas as pd
from PIL import Image
from diffusers import MarigoldDepthPipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from geographiclib.geodesic import Geodesic
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN ---
JSON_FILE = "data.json" # Tu archivo de datos
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- FUNCIONES AUXILIARES ---
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlam = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlam/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def calcular_azimut_real(lat1, lon1, lat2, lon2):
    """Calcula el ángulo real entre dos puntos para aislar el error de distancia"""
    g = Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)
    return g['azi1']

# ==========================================
# 1. CARGAR MODELO (SOLO UNA VEZ)
# ==========================================
print(f"🚀 Cargando Marigold (Optimizado) en {DEVICE}...")
try:
    pipe = MarigoldDepthPipeline.from_pretrained(
        "prs-eth/marigold-lcm-v1-0",
        torch_dtype=torch.float16
    ).to(DEVICE)
    pipe.enable_xformers_memory_efficient_attention()
except:
    print(" Aviso: Xformers no disponible (correrá un poco más lento, pero igual funciona).")


def probar_marigold_optimizado():
    with open(JSON_FILE, 'r') as f:
        experimentos = json.load(f)

    resultados = []
    print(f"🔬 Iniciando análisis de profundidad en {len(experimentos)} imágenes...")

    for exp in experimentos:
        img_name = exp['imagen']
        cam_gps = exp['camara_gps']
        gcps = exp['gcps']
        checkpoints = exp['checkpoints']

        print(f"\n📸 Procesando Imagen: {img_name}")

        try:
            # A. INFERENCIA IA
            img_pil = Image.open(img_name).convert("RGB")

            # Usamos ensemble_size=10 para máxima calidad (aunque tarde unos segundos más)
            pipe_out = pipe(
                img_pil,
                num_inference_steps=4,
                ensemble_size=10,
                match_input_resolution=True,
                batch_size=1
            )

            # Procesar el array de profundidad
            depth_map = pipe_out.prediction
            if not isinstance(depth_map, np.ndarray): depth_map = np.array(depth_map)
            depth_map = np.squeeze(depth_map)

            # Normalización robusta (0 a 1)
            depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min() + 1e-8)

            # Guardar imagen para que veas qué detectó
            plt.imsave(f"Debug_Depth_{img_name}", depth_map, cmap='Spectral')

            # B. ENTRENAMIENTO DE ESCALA (REGRESIÓN LINEAL CON GCPS)
            X_ia = []   # Valor que ve la IA (0-1)
            y_real = [] # Metros reales

            for gcp in gcps:
                u, v = gcp['pixel']
                # Distancia real del GCP a la cámara
                d_real = haversine_distance(cam_gps[0], cam_gps[1], gcp['geo'][0], gcp['geo'][1])

                # Valor de la IA en ese pixel
                v_safe = int(np.clip(v, 0, depth_map.shape[0]-1))
                u_safe = int(np.clip(u, 0, depth_map.shape[1]-1))
                val_ia = float(depth_map[v_safe, u_safe])

                X_ia.append(val_ia)
                y_real.append(d_real)

            # Ajuste Matemático (Encontrar la mejor recta que une los puntos)
            X_train = np.array(X_ia).reshape(-1, 1)
            y_train = np.array(y_real)

            reg = LinearRegression().fit(X_train, y_train)
            r2 = r2_score(y_train, reg.predict(X_train))

            scale = reg.coef_[0]
            offset = reg.intercept_

            print(f"   📐 Calibración: Distancia = ({scale:.1f} * Valor_IA) + {offset:.1f}")
            print(f"   📊 Coherencia de la IA (R2): {r2:.4f} (1.0 es perfecto, <0.5 es malo)")

            # C. EVALUACIÓN EN PUNTOS DE PRUEBA
            for i, chk in enumerate(checkpoints):
                u, v = chk['pixel']
                geo_true = chk['geo_true']

                # 1. Distancia Real (La verdad)
                dist_real = haversine_distance(cam_gps[0], cam_gps[1], geo_true[0], geo_true[1])

                # 2. Distancia IA (La predicción calibrada)
                v_s = int(np.clip(v, 0, depth_map.shape[0]-1))
                u_s = int(np.clip(u, 0, depth_map.shape[1]-1))
                val_pixel = float(depth_map[v_s, u_s])

                dist_predicha = (scale * val_pixel) + offset
                if dist_predicha < 0: dist_predicha = 0 # Física imposible

                # 3. Error Absoluto en Metros
                error_distancia = abs(dist_real - dist_predicha)

                resultados.append({
                    "Imagen": img_name,
                    "Punto_ID": i+1,
                    "Distancia_Real_m": round(dist_real, 2),
                    "Distancia_IA_m": round(dist_predicha, 2),
                    "Error_Metros": round(error_distancia, 2),
                    "Error_Relativo_%": round((error_distancia/dist_real)*100, 2)
                })

        except Exception as e:
            print(f" Error procesando {img_name}: {e}")
            continue

    return pd.DataFrame(resultados)

# --- EJECUTAR ---
df_solo_marigold = probar_marigold_optimizado()

print("\n📊 RESULTADOS OPTIMIZADOS MARIGOLD:")
display(df_solo_marigold)

if not df_solo_marigold.empty:
    mae = df_solo_marigold['Error_Metros'].mean()
    mre = df_solo_marigold['Error_Relativo_%'].mean()
    print(f"\n Error Medio Absoluto (MAE): {mae:.2f} metros")
    print(f" Error Relativo Promedio:    {mre:.2f} %")

🚀 Cargando Marigold (Optimizado) en cuda...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

...

Diffusion steps...:   0%|          | 0/4 [00:00<?, ?it/s]

Diffusion steps...:   0%|          | 0/4 [00:00<?, ?it/s]

   📐 Calibración: Distancia = (26458.6 * Valor_IA) + -14306.2
   📊 Coherencia de la IA (R2): 0.8730 (1.0 es perfecto, <0.5 es malo)

📊 RESULTADOS OPTIMIZADOS MARIGOLD:


,Imagen,Punto_ID,Distancia_Real_m,Distancia_IA_m,Error_Metros,Error_Relativo_%
0,Imagen_1.PNG,1,403.13,0.00,403.13,100.00
1,Imagen_1.PNG,2,981.56,617.91,363.65,37.05
2,Imagen_1.PNG,3,6483.34,7951.93,1468.59,22.65
3,Imagen_1.PNG,4,870.93,250.47,620.46,71.24
4,Imagen_1.PNG,5,4701.87,6776.13,2074.26,44.12
5,Imagen_1.PNG,6,7875.72,6923.11,952.62,12.10
6,Imagen_1.PNG,7,494.23,0.00,494.23,100.00
7,Imagen_1.PNG,8,1686.57,882.46,804.11,47.68
8,Imagen_1.PNG,9,1164.19,0.00,1164.19,100.00
9,Imagen_1.PNG,10,4572.89,4938.95,366.06,8.01



 Error Medio Absoluto (MAE): 835.02 metros
 Error Relativo Promedio:    68.72 %
